In [1]:
# import all libraries
from plotly import graph_objects as go
import pandas as pd
import dash
from dash.dependencies import Input, Output
from dash import Dash, html, dcc 
from urllib.request import urlopen
import json
from plotly.subplots import make_subplots

In [2]:
# Load Veggie Data
df = pd.read_csv("kv21_trimmet_98.csv",
                 dtype={"fips": str})
df = df.fillna(0) # replace NA values with 0
df_nameIndex = df.set_index("Navn")

## Lollipop 4 eva

In [44]:
df_temp = df_nameIndex.iloc[:10]
tickvals_ = list(range(len(df_temp)))
ticktext_ = list(df_temp.index)
ticktext_

['Flemming Jørgensen',
 'Nicolai Kofod-Jensen',
 'Billal Zahoor',
 'Inge Dahn',
 'Michael Jonasen',
 'Adam Samy',
 'Jeppe Milthers',
 'Lars Bremer',
 'Niels Christian Dahl',
 'Vibeke Højbjerg']

In [37]:
def Marker_size(lenght):
    markerList = []
    for i in range(lenght):
        markerList.extend([0,10])
    return markerList

In [36]:
def UpdateHeadline(value):
    return f"Sammenlign kandidatter fra {value}"

[0, 10, 0, 10, 0, 10, 0, 10, 0, 10, 0, 10, 0, 10, 0, 10]

In [67]:
fig = go.Figure()
df_temp = df_nameIndex.iloc[:3]

tickvals_ = list(range(len(df_temp)))

ticktext_ = list(df_temp.index)


veggieGreen = 'rgb(5,122,87)' # Dark-green for the vegetarian color option !!! Change for real color
markerSize = Marker_size(len(df_temp))


for i, mean in enumerate(df_temp["Score"]):
    fig.add_trace(go.Scatter(x=[i,i],y=[0,mean], 
                             marker={"color":veggieGreen,"size":markerSize},
                            line=go.scatter.Line(color=veggieGreen),
                            showlegend=False))



fig.update_layout(
    xaxis = dict(
        tickmode = "array",
        tickvals = tickvals_,
        ticktext = ticktext_),
    xaxis_range=[-1,len(df_temp)]
    )

    
fig.show()

In [7]:
fig.add_trace(go.Scatter(x=data["Time"], y=data["OD"], mode='markers', 
                         marker=dict(color=data["C-source"], size=data["C:A 1 ratio"])))